#**5.4 - Exploração dos dados com SQL**

In [0]:
#!pip install -U pandasql
import pandas as pd
import pandasql

In [14]:
df = pd.read_csv('ovni_data.csv')

#Excluindo coluna Indesejada
del df['Unnamed: 0']

#df.columns = ['Date/Time','City','State','Shape','Duration','Summary','Posted']

linhas = df['Date / Time'].size

print("Linhas: ",linhas)

Linhas:  99705


##**Ordenando relatos por estado e em ordem decrescente** 
**Removendo campos em branco**

In [0]:
estados = """
   SELECT State, COUNT(State) as qtd_rel FROM df WHERE State != ' ' GROUP BY State ORDER BY COUNT(State) desc ;
 """
result_consulta = pandasql.sqldf(estados)

##**Análise dos estados dos Estados Unidos**

In [0]:
#Cria um array com os estados da coluna estado.

lista_usa = ['AK','AL','AR','AZ','CA','CO','CT','DE','FL','GA',
             'HI','IA','ID','IL','IN','KS','KY','LA','MA','MD',
             'ME','MI','MN','MO','MS','MT','NC','ND','NE','NH',
             'NJ','NM','NV','NY','OH','OK','OR','PA','RI','SC',
             'SD','TN','TX','UT','VA','VT','WA','WI','WV','WY']

lista = pd.DataFrame(lista_usa, columns=['Estados_Unidos'])

In [0]:
consulta_est_validos = '''
  SELECT df.* FROM df, lista
  WHERE State = Estados_Unidos
'''

limit_usa = pandasql.sqldf(consulta_est_validos)

##**Relatos das cidades que possuem 10 relatos ou mais**

In [0]:
cidades_agrup = """
    SELECT State, Shape, City, COUNT(*) as total_posts FROM limit_usa GROUP BY City;
  """
result_cidades_agrup = pandasql.sqldf(cidades_agrup)

nova_query_usa = """
    SELECT State, Shape, City, total_posts 
    FROM result_cidades_agrup 
    WHERE City != 'unknown' AND total_posts >= 10 
    ORDER BY total_posts desc;
    """
sql_cidades_usa = pandasql.sqldf(nova_query_usa)

##**Por que será que essa é a cidade que possui mais relatos?**

###**Phoenix**	- No dia 13 de março de 1997, o céu do estado do Arizona e de Nevada, encheram de luzes, milhares de pessoas presenciaram o ocorrido, as pessoas afirmaram ver algo em formato triangular vagando pelo céu na cidade de Phoenix, capital do Arizona e cidade com mais relatos na análise de todos os casos de aparecimento de OVNI’s, o próprio governador presenciou e afirmou que seria um “objeto de outro mundo". Milhares de pessoas registraram essas aparições isso explica ser um dos locais com mais relatos.

####Fonte: https://aventurasnahistoria.uol.com.br/noticias/reportagem/luzes-phoenix-bizarras-aparicoes-de-ovnis-nos-estados-unidos.phtml

##**Fazer uma query exclusiva para o estado com maior número de relatos, buscando cidades que possuam um número superior a 10 relatórios. Enfatizar a cidade, a quantidade de relatos e formato do objeto não identificado.**

In [0]:
estados_usa_mais_rel = """
    SELECT State, MAX(qtd_rel) as qtd_rel FROM result_consulta;
    """
sql_estados = pandasql.sqldf(estados_usa_mais_rel)

cid_mais_rel = """
  SELECT sql_cidades_usa.City, sql_cidades_usa.total_posts, sql_cidades_usa.Shape 
  FROM sql_estados, sql_cidades_usa 
  WHERE sql_cidades_usa.State = sql_estados.State AND sql_cidades_usa.total_posts > 10
"""

result_final_cidades = pandasql.sqldf(cid_mais_rel)

result_final_cidades.to_csv('Tentativaeerro.csv')

###Retirando campos em branco e campos "Unknown" da coluna Shape

In [0]:
estados_usa_mais_rel = """
    SELECT State, MAX(qtd_rel) as qtd_rel FROM result_consulta;
    """
sql_estados = pandasql.sqldf(estados_usa_mais_rel)

cid_mais_rel = """
  SELECT sql_cidades_usa.City, sql_cidades_usa.total_posts, sql_cidades_usa.Shape 
  FROM sql_estados, sql_cidades_usa 
  WHERE sql_cidades_usa.State = sql_estados.State AND sql_cidades_usa.total_posts > 10 AND sql_cidades_usa.Shape NOT IN (' ', 'Unknown')
"""

result_final_cidades = pandasql.sqldf(cid_mais_rel)